In [37]:
import calendar
import datetime as dt
import warnings

import matplotlib.pyplot as plt
import pandas as pd
from pandas.errors import SettingWithCopyWarning
import plotly.express as px
import plotly.graph_objects as go

In [3]:
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [4]:
BUG_CSV_PATH = "./../datasets/bug.csv"
INSIDER_CSV_PATH = "./../datasets/insider_media.csv"
VOLYNNEWS_CSV_PATH = "./../datasets/volynnews.csv"

RESULT_DIR = "./../results"

In [5]:
TOPICS = [
    {
        "name": "загиблі волиняни на фронті",
        "keywords": [
            "загинув",
            "волинянин",
            "загинув+волинянин"
        ]
    },
    {
        "name": "обстріли, їх наслідки",
        "keywords": [
            "обстріл",
            "обстріл+внаслідок",
            "постраждалі",
            "критична інфраструктура",
            "шахеди",
            "бпла",
            "укриття"
        ]
    },
    {
        "name": "державна зрада (чиновники та громадяни україни)",
        "keywords": [
            "держзрада",
            "чиновник+обвинувачення",
            "звинувачення+держзрада"
        ]
    },
    {
        "name": "ситуація на фронті, зведення генштабу",
        "keywords": [
            "ситуація фронт",
            "звередення",
            "генштаб"
        ]
    },
    {
        "name": "ситуація в регіоні (волинь)",
        "keywords": [
            "ситуація+регіон"
        ]
    },
    {
        "name": "роль та участь, допомога волині на фронті. благодійність, допомога воїнам (волинь)",
        "keywords": [
            "відправили+допомога",
            "волонтери"
        ]
    },
    {
        "name": "злочини та звірства росіян, суди над ними",
        "keywords": [
            "злочин+росіяни",
            "злочин",
            "росіяни"
        ]
    },
    {
        "name": "ситуація російської армії",
        "keywords": [
            "росіяни+російський фронт",
            "дії+росіяни+фронт"
        ]
    },
    {
        "name": "ситуація в окупованому маріуполі, азовсталі ",
        "keywords": [
            "окупація+маріуполь",
            "азов",
            "азовсталь",
            "бійці+азов",
            "полон+азов"
        ]
    },
    {
        "name": "інформація про переселенців біженців, внутрішньо переміщених осіб",
        "keywords": [
            "біженці",
            "біженець",
            "біженка",
            "переселенці",
            "впо",
            "внутрішньо переміщена особа"
        ]
    },
    {
        "name": "окуповані території (ситуація, події)",
        "keywords": [
            "окуповані",
            "окупація",
            "режим окупації"
        ]
    },
    {
        "name": "зовнішня політика. передача озброєння, підтримка світу",
        "keywords": [
            "підтримка+європейський союз",
            "підтримка+сполучені штати америки",
            "підтримка+країна",
            "передача+озброєння",
            "передати+озброєння",
            "гуманітарна допомога",
            "гуманітарна підтримка"
        ]
    },
    {
        "name": "накладання санкцій, ембарго на російську федерацію ",
        "keywords": [
            "санкція",
            "ембарго",
            "накладено+санкції",
            "накладено+ембарго",
            "заборона+росія"
        ]
    },
    {
        "name": "проблеми зі світлом внаслідок обстрілів",
        "keywords": [
            "блекаут",
            "критична інфраструктура+обстріл."
        ]
    },
    {
        "name": "дія, зв'язок, цифрові трансформації",
        "keywords": [
            "зв’язок+проблеми",
            "єпідтримка"
        ]
    },
    {
        "name": "інформаційна війна, пропаганда рф, спростування фейків",
        "keywords": [
            "іпсо",
            "пропаганда+рф",
            "фейк",
            "російський+фейк",
            "поширюють+фейк",
            "поширюють+пропаганда"
        ]
    },
    {
        "name": "новини з тилу (всеукраїнські)",
        "keywords": [
            "війна+україна",
            "повномасштабне вторгнення+україна"
        ]
    },
    {
        "name": "білоруський фронт, ситуація з ймовірним наступом, прикордонні райони з білоруссю",
        "keywords": [
            "наступ+білорусь",
            "наступ+білоруський напрямок"
        ]
    },
    {
        "name": "героїзм українців (волинян в т.ч.)",
        "keywords": [
            "героїчний+вчинок"
        ]
    },
    {
        "name": "освіта",
        "keywords": [
            "навчання+війна"
        ]
    },
    {
        "name": "мобілізація в україні (волинь у т.ч.)",
        "keywords": [
            "мобілізація"
        ]
    },
    {
        "name": "переговори, обміни",
        "keywords": [
            "переговори",
            "обмін+полоненими"
        ]
    },
    {
        "name": "мобілізація в рф",
        "keywords": [
            "мобілізація+росіян",
            "мобілізація+рф"
        ]
    },
    {
        "name": "акції, вшанування, урочистості, дозвілля (волинь у т.ч.)",
        "keywords": [
            "проведено+акцію",
            "проведено+урочисте."
        ]
    },
    {
        "name": "робота державних та приватних структур",
        "keywords": [
            "сесія+війна",
            "чергова сесія+питання+війна"
        ]
    },
    {
        "name": "діти",
        "keywords": [
            "діти",
            "діти+обстріл",
            "діти+війна"
        ]
    },
    {
        "name": "героїчні, незвичайні історії",
        "keywords": [
            "поділився+історією",
            "дивовижна+історія",
            "вдалося+врятуватися",
            "дивом+врятуватися"
        ]
    }
]

EXTENDED_KEYWORDS = [
    {
        "main_keyword": "війна",
        "additional_keywords": [
            "війні",
            "війною",
            "війни",
            "війна"
        ]
    },
    {
        "main_keyword": "повномасштабне вторгнення",
        "additional_keywords": [
            "повномасштабного вторгнення",
            "повномасштабне вторгнення"
        ]
    },
    {
        "main_keyword": "обстріл",
        "additional_keywords": [
            "обстрілів",
            "обстрілу",
            "обстріл"
        ]
    },
    {
        "main_keyword": "загинув",
        "additional_keywords": [
            "загинула",
            "загинули",
            "загинув"
        ]
    },
    {
        "main_keyword": "ЗСУ",
        "additional_keywords": [
            "ЗСУ"
        ]
    },
    {
        "main_keyword": "шахеди",
        "additional_keywords": [
            "шахедів",
            "шахеди"
        ]
    },
    {
        "main_keyword": "БПЛА",
        "additional_keywords": [
            "БПЛА"
        ]
    },
    {
        "main_keyword": "наступ",
        "additional_keywords": [
            "наступу",
            "наступом",
            "наступ"
        ]
    },
    {
        "main_keyword": "воєнний стан",
        "additional_keywords": [
            "воєнного стану",
            "воєнним станом",
            "воєнний стан"
        ]
    },
    {
        "main_keyword": "країна-агресор",
        "additional_keywords": [
            "країна-агресорка",
            "країни-агресорки",
            "країною-агресором",
            "країна-агресор"
        ]
    },
    {
        "main_keyword": "бойові дії",
        "additional_keywords": [
            "бойових дій",
            "бойові дії"
        ]
    },
    {
        "main_keyword": "військовий",
        "additional_keywords": [
            "військового",
            "військових",
            "військовий"
        ]
    },
    {
        "main_keyword": "ворог",
        "additional_keywords": [
            "ворога",
            "ворогу",
            "ворогами",
            "ворог"
        ]
    },
    {
        "main_keyword": "армія",
        "additional_keywords": [
            "армія"
        ]
    },
    {
        "main_keyword": "росіяни",
        "additional_keywords": [
            "росіян",
            "росіяни"
        ]
    },
    {
        "main_keyword": "фронт",
        "additional_keywords": [
            "фронті",
            "фронт"
        ]
    },
    {
        "main_keyword": "санкції",
        "additional_keywords": [
            "санкцій",
            "санкціями",
            "санкції"
        ]
    },
    {
        "main_keyword": "укриття",
        "additional_keywords": [
            "укритті",
            "укриття"
        ]
    },
    {
        "main_keyword": "пропаганда",
        "additional_keywords": [
            "пропаганди",
            "пропагандою",
            "пропаганда"
        ]
    },
    {
        "main_keyword": "окупанти",
        "additional_keywords": [
            "окупантам",
            "окупантами",
            "окупанти"
        ]
    },
    {
        "main_keyword": "біженці",
        "additional_keywords": [
            "біженців",
            "біженцями",
            "біженці"
        ]
    },
    {
        "main_keyword": "внутрішньо-переміщена особа",
        "additional_keywords": [
            "ВПО",
            "внутрішньо-переміщених осіб",
            "внутрішньо-переміщена особа"
        ]
    },
    {
        "main_keyword": "гуманітарна допомога",
        "additional_keywords": [
            "гуманітарною допомогою",
            "гуманітарну допомогу",
            "підтримка",
            "підтримку",
            "гуманітарна допомога"
        ]
    },
    {
        "main_keyword": "мобілізація",
        "additional_keywords": [
            "мобілізацією",
            "мобілізації",
            "мобілізація"
        ]
    },
    {
        "main_keyword": "Азов",
        "additional_keywords": [
            "Азов"
        ]
    },
    {
        "main_keyword": "державна зрада",
        "additional_keywords": [
            "державній зраді",
            "державною зрадою",
            "державна зрада"
        ]
    },
    {
        "main_keyword": "ракети",
        "additional_keywords": [
            "ракети"
        ]
    },
    {
        "main_keyword": "загиблі",
        "additional_keywords": [
            "загиблих",
            "загиблими",
            "загиблі"
        ]
    },
    {
        "main_keyword": "постраждалі",
        "additional_keywords": [
            "постраждалих",
            "постраждалими",
            "постраждалі"
        ]
    }
]

In [6]:
bug_df = pd.read_csv(BUG_CSV_PATH).drop_duplicates()
bug_df["source"] = "bug.org.ua"

insider_df = pd.read_csv(INSIDER_CSV_PATH).drop_duplicates()
insider_df["source"] = "insider-media.net"

volynnews_df = pd.read_csv(VOLYNNEWS_CSV_PATH).drop_duplicates()
volynnews_df["source"] = "volynnews.com"

In [7]:
sources_df = pd.concat([bug_df, insider_df, volynnews_df])

In [8]:
topics_df = sources_df.copy()

for topic in TOPICS:
    topics_df[topic["name"]] = 0

In [9]:
topics_df.shape

(58290, 32)

In [10]:
keywords_df = sources_df.copy()

for keyword in EXTENDED_KEYWORDS:
    keywords_df[keyword["main_keyword"]] = 0

In [11]:
keywords_df.shape

(58290, 34)

## Topics and Keywords Mentions Caclculation

In [12]:
def check_topic_keyword_mention(keyword: str, text: str) -> bool:
    found = False
    for keyword in keyword.split("+"):
        if keyword not in text:
            break
    else:
        found = True
    return found    

In [13]:
def get_keyword_mentions_number(keywords: list[str], text: str) -> bool:
    mentions = 0
    for keyword in keywords:
        mentions += text.lower().count(keyword.lower())
    return mentions

In [14]:
def chech_topics_mention(row: pd.Series, topics: list[dict]) -> pd.Series:
    joinded_text = str(row["title"]) + str(row["text"])
    for topic in topics:
        for keyword in topic["keywords"]:
            if check_topic_keyword_mention(keyword=keyword, text=joinded_text.lower()):
                row[topic["name"]] = 1
                break
    return row

In [15]:
def chech_total_keywords_mentions(row: pd.Series, keywords: list[dict]) -> pd.Series:
    joinded_text = str(row["title"]) + str(row["text"])
    for item in keywords:
        main_keyword = item["main_keyword"]
        row[main_keyword] = get_keyword_mentions_number(keywords=item["additional_keywords"], text=joinded_text)
    return row

In [16]:
%%time

topics_df = topics_df.apply(lambda row: chech_topics_mention(row=row, topics=TOPICS), axis=1)

CPU times: user 56.2 s, sys: 430 ms, total: 56.7 s
Wall time: 57.1 s


In [17]:
topics_df.shape

(58290, 32)

In [18]:
%%time

keywords_df = keywords_df.apply(lambda row: chech_total_keywords_mentions(row=row, keywords=EXTENDED_KEYWORDS), axis=1)

CPU times: user 1min 11s, sys: 412 ms, total: 1min 11s
Wall time: 1min 12s


In [19]:
keywords_df.shape

(58290, 34)

## Topics Stats Calculation

In [20]:
def generate_topic_stats_df(topics_df: pd.DataFrame, topics: list[dict], source: str) -> pd.DataFrame:
    topic_names = [topic["name"] for topic in TOPICS]
    
    source_df = topics_df[topics_df["source"] == source]
    source_df['date'] = pd.to_datetime(source_df['date'], errors='coerce')
    source_df = source_df[~source_df.date.isnull()]
    
    filtered_df = source_df[(source_df['date'] >= "2022-03-01") & (source_df['date'] < "2023-09-01")]
    filtered_df['total_mentions']= filtered_df[topic_names].sum(axis=1)
    
    grouped_df = filtered_df.groupby(pd.Grouper(key='date', freq='M'))
    months_dfs = [group for _,group in grouped_df]
    
    topics_data = []

    for topic_name in topic_names:
        total_mentions = filtered_df[topic_name].sum()
        overall_topics_mentions = filtered_df["total_mentions"].sum()
                
        month_values = [df[topic_name].sum() for df in months_dfs]

        topics_data.append(
            {
                "Тема": topic_name, 
                "Кількість згадувань": total_mentions, 
                "Середня кількість згадувань за місяць": round(sum(month_values) / len(month_values), 2),
                "Загальний відсоток згадувань": round(total_mentions / overall_topics_mentions * 100, 2)
            }
        )
    return pd.DataFrame(topics_data).sort_values(by=["Кількість згадувань"], ascending=False)

In [21]:
bug_topics_df = generate_topic_stats_df(topics_df=topics_df, topics=TOPICS, source="bug.org.ua")

In [22]:
insider_topics_df = generate_topic_stats_df(topics_df=topics_df, topics=TOPICS, source="insider-media.net")

In [23]:
volynnews_topics_df = generate_topic_stats_df(topics_df=topics_df, topics=TOPICS, source="volynnews.com")

In [24]:
bug_topics_df.to_excel(f"{RESULT_DIR}/topics_bug.xlsx", index=False)
insider_topics_df.to_excel(f"{RESULT_DIR}/topics_insider.xlsx", index=False)
volynnews_topics_df.to_excel(f"{RESULT_DIR}/topics_volynnews.xlsx", index=False)

## Keyword Stats Calculation

In [24]:
def generate_keyword_stats_df(keywords_df: pd.DataFrame, keywords: list[dict], source: str) -> pd.DataFrame:    
    source_df = keywords_df[keywords_df["source"] == source]
    source_df['date'] = pd.to_datetime(source_df['date'], errors='coerce')
    source_df = source_df[~source_df.date.isnull()]
    
    keywords = [item["main_keyword"] for item in keywords]
    
    filtered_df = source_df[(source_df['date'] >= "2022-03-01") & (source_df['date'] < "2023-09-01")]
    filtered_df['total_mentions']= filtered_df[keywords].sum(axis=1)
    
    grouped_df = filtered_df.groupby(pd.Grouper(key='date', freq='M'))
    months_dfs = [group for _,group in grouped_df]
    
    keywords_data = []
    
    for keyword in keywords:
        total_mentions = filtered_df[keyword].sum()
        overall_keywords_mentions = filtered_df["total_mentions"].sum()
                
        month_values = [df[keyword].sum() for df in months_dfs]

        keywords_data.append(
            {
                "Ключове слово": keyword, 
                "Кількість згадувань": total_mentions,
                "Частота згадувань на 1000 статей": round(total_mentions / len(source_df) * 1000),
                "Середня кількість згадувань за місяць": round(sum(month_values) / len(month_values), 2),
                "Загальний відсоток згадувань": round(total_mentions / overall_keywords_mentions * 100, 2)
            }
        )
    return pd.DataFrame(keywords_data).sort_values(by=["Кількість згадувань"], ascending=False)

In [50]:
def build_keywords_pie_chart(source_df: pd.DataFrame, source: str) -> None:        
    fig = px.pie(
        source_df,
        values='Загальний відсоток згадувань',
        names='Ключове слово',
        title=f'Чарт ключових слів ({source})',
        
    )
    fig.update_layout(
        autosize=False,
        title_x=0.48,
        width=1000,
        height=800,
    )   
    return fig

In [27]:
bug_keywords_df = generate_keyword_stats_df(keywords_df=keywords_df, keywords=EXTENDED_KEYWORDS, source="bug.org.ua")

In [28]:
insider_keywords_df = generate_keyword_stats_df(keywords_df=keywords_df, keywords=EXTENDED_KEYWORDS, source="insider-media.net")

In [29]:
volynnews_keywords_df = generate_keyword_stats_df(keywords_df=keywords_df, keywords=EXTENDED_KEYWORDS, source="volynnews.com")

In [30]:
bug_keywords_df.to_excel(f"{RESULT_DIR}/keywords_bug.xlsx", index=False)
insider_keywords_df.to_excel(f"{RESULT_DIR}/keywords_insider.xlsx", index=False)
volynnews_keywords_df.to_excel(f"{RESULT_DIR}/keywords_volynnews.xlsx", index=False)

In [51]:
p = build_keywords_pie_chart(source_df=bug_keywords_df, source="bug.org.ua")
p.write_image(f'{RESULT_DIR}/keywords_stats_pie_bug.png')

p = build_keywords_pie_chart(source_df=insider_keywords_df, source="insider-media.net")
p.write_image(f'{RESULT_DIR}/keywords_stats_pie_insider.png')

p = build_keywords_pie_chart(source_df=volynnews_keywords_df, source="volynnews.com")
p.write_image(f'{RESULT_DIR}/keywords_stats_pie_volynnews.png')

## Source Analysis

In [53]:
def get_printed_period_info(date: str) -> tuple:
    date = date.split(" ")[3][:7]
    month_map = {
        "01": "Січень",
        "02": "Лютий",
        "03": "Березень",
        "04": "Квітень",
        "05": "Травень",
        "06": "Червень",
        "07": "Липень",
        "08": "Серпень",
        "09": "Вересень",
        "10": "Жовтень",
        "11": "Листопад",
        "12": "Грудень",
    }
    year, month = date.split("-")
    month_days = calendar.monthrange(int(year), int(month))[1]
    return date, f"{month_map[month]} {year}", month_days

In [54]:
def generate_source_stats_df(sources_df: pd.DataFrame, source: str) -> pd.DataFrame:    
    source_df = sources_df[sources_df["source"] == source]
    source_df['date'] = pd.to_datetime(source_df['date'], errors='coerce')
    source_df = source_df[~source_df.date.isnull()]
    
    filtered_df = source_df[(source_df['date'] >= "2022-03-01") & (source_df['date'] < "2023-09-01")]
    
    grouped_df = filtered_df.groupby(pd.Grouper(key='date', freq='M'))
    month_dfs = [group for _,group in grouped_df]
    
    source_data = []
    
    for month_df in month_dfs:
        date = str(month_df.head(1)["date"])
        period_date, period, month_days = get_printed_period_info(date)
        source_data.append(
            {
                "Період": period,
                "Дата": period_date,
                "Кількість публікацій": len(month_df),
                "Середня кількість публікацій в день": round(len(month_df) / month_days),
            }
        )
    return pd.DataFrame(source_data)

In [76]:
def build_source_stats_diagram(source_df: pd.DataFrame, source: str) -> None:
    fig, ax = plt.subplots()
    plt.xticks(rotation=90)

    periods = source_df["Період"].values
    publications = source_df["Кількість публікацій"].values

    ax.bar(periods, publications)
    ax.set_xlabel('Період')
    ax.set_ylabel('Кількість публікацій')
    ax.set_title(f'Діаграма публікацій ({source})')
    
    return fig

In [55]:
bug_stats_df = generate_source_stats_df(sources_df=sources_df, source="bug.org.ua")

In [69]:
insider_stats_df = generate_source_stats_df(sources_df=sources_df, source="insider-media.net")

In [70]:
volynnews_stats_df = generate_source_stats_df(sources_df=sources_df, source="volynnews.com")

In [35]:
bug_stats_df.to_excel(f"{RESULT_DIR}/source_stats_bug.xlsx", index=False)
insider_stats_df.to_excel(f"{RESULT_DIR}/source_stats_insider.xlsx", index=False)
volynnews_stats_df.to_excel(f"{RESULT_DIR}/source_stats_volynnews.xlsx", index=False)

In [78]:
d = build_source_stats_diagram(source_df=bug_stats_df, source="bug.org.ua")
d.savefig(f'{RESULT_DIR}/source_stats_diagram_bug.png', bbox_inches='tight')
plt.close(d)

d = build_source_stats_diagram(source_df=insider_stats_df, source="insider-media.net")
d.savefig(f'{RESULT_DIR}/source_stats_diagram_insider.png', bbox_inches='tight')
plt.close(d)

d = build_source_stats_diagram(source_df=volynnews_stats_df, source="volynnews.com")
d.savefig(f'{RESULT_DIR}/source_stats_diagram_volynnews.png', bbox_inches='tight')
plt.close(d)